<a href="https://colab.research.google.com/github/amarlatchireddy/llm_chatbot_/blob/main/text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Install Langchain and Google Gemini related libraries**

In [ ]:
# !pip install -q -U langchain
# !pip install -q -U google-generativeai
# !pip install -q -U langchain-google-genai

# Get Gemini API Key from Secrets
Set GEMINI_KEY secret key at Google Colab and get that here to runn Google Gemini LLM. You can get Google Gemini Key from following link https://makersuite.google.com/app/apikey

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

Copy chinhook.db to your Google Drive

Now, chinhook.db is in our directory and we can interface with it using the SQLAlchemy-driven SQLDatabase class.

Location of chinhook.db
/content/drive/MyDrive/Colab Notebooks/chinook.db

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////content/drive/MyDrive/chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Chain
Let’s create a simple chain that takes a question, turns it into a SQL query, executes the query, and uses the result to answer the original question.

# Convert question to SQL query
The first step in a SQL chain or agent is to take the user input and convert it to a SQL query. LangChain comes with a built-in chain for this: create_sql_query_chain

In [ ]:
import re
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True, temperature=0.0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})

# Generate regex pattern to remove prefix and suffix
regex_pattern = r'^```sql\n|\n```$'

# Remove prefix and suffix using regex to get proper SQL query
modified_response = re.sub(regex_pattern, '', response)
print(modified_response)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


SELECT COUNT(*) AS "Number of Employees"
FROM employees;


In [ ]:
db.run(modified_response)

'[(8,)]'

In [ ]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#Execute SQL query
Now that we’ve generated a SQL query, we’ll want to execute it.

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

execute_query = QuerySQLDataBaseTool(db=db)

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the {top_k} answer.
Use the following format:

Question: "Question here"
"SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Provide SQL query as simple string without any markdown.

Question: {input}'''
prompt = PromptTemplate.from_template(template)

write_query = create_sql_query_chain(llm, db, prompt)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'[(8,)]'

#Answer the question
Now that we’ve got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer.

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
{query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many artists are there"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'There are 275 artists.'

# Agents
LangChain has an SQL Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
It can answer questions that require multiple dependent queries.
It will save tokens by only considering the schema from relevant tables.
To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

agent_executor = create_sql_agent(llm,
                                  toolkit=SQLDatabaseToolkit(db=db, llm=llm),
                                  agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                  verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...


/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: sql_db_list_tables
Action Input: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: sql_db_query
Action Input: SELECT country, SUM(total) AS total_sales FROM invoices GROUP BY country ORDER BY total_sales DESCError: (sqlite3.OperationalError) no such column: country
[SQL: SELECT country, SUM(total) AS total_sales FROM invoices GROUP BY country ORDER BY total_sales DESC]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

...

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Action: sql_db_query
Action Input: SELECT c.country, SUM(i.total) AS total_sales FROM customers c JOIN invoices i ON c.customerid = i.customerid GROUP BY c.country ORDER BY total_sales DESC[('USA', 523.0600000000004), ('Canada', 303.96), ('France', 195.09999999999994), ('Brazil', 190.1), ('Germany', 156.48), ('United Kingdom', 112.85999999999999), ('Czech Republic', 90.24), ('Portugal', 77.24), ('India', 75.25999999999999), ('Chile', 46.62), ('Ireland', 45.62), ('Hungary', 45.62), ('Austria', 42.62), ('Finland', 41.620000000000005), ('Netherlands', 40.62), ('Norway', 39.62), ('Sweden', 38.620000000000005), ('Poland', 37.620000000000005), ('Italy', 37.620000000000005), ('Denmark', 37.620000000000005), ('Australia', 37.620000000000005), ('Argentina', 37.620000000000005), ('Spain', 37.62), ('Belgium', 37.62)]

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Final Answer: The country with the highest total sales is the USA, with $523.06 in total sales.

> Finished chain.


{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The country with the highest total sales is the USA, with $523.06 in total sales.'}